<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Decision Trees


## Objectives


-   Membangaun klasifikasi model dengan Algoritma Decision Tree


Algoritma machine learning, Decision Tree. Anda akan menggunakan algoritme klasifikasi ini untuk membuat model historical data 
pasien, dan respons mereka terhadap obat yang berbeda. Kemudian Anda menggunakan yang terlatih decision tree untuk memprediksi kelas pasien yang tidak diketahui,atau untuk menemukan obat yang tepat untuk pasien baru.


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#about_dataset">About the dataset</a></li>
        <li><a href="#pre-processing">Pre-processing</a></li>
        <li><a href="#setting_up_tree">Setting up the Decision Tree</a></li>
        <li><a href="#modeling">Modeling</a></li>
        <li><a href="#prediction">Prediction</a></li>
        <li><a href="#evaluation">Evaluation</a></li>
        <li><a href="#visualization">Visualization</a></li>
    </ol>
</div>
<br>
<hr>


Import Libraries:

<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
    <li> <b>DecisionTreeClassifier</b> from <b>sklearn.tree</b> </li>
</ul>


In [30]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

<div id="about_dataset">
    <h2>About the dataset</h2>
    Bayangkan Anda adalah seorang peneliti medis yang mengumpulkan data untuk sebuah penelitian. Anda telah mengumpulkan data tentang sekumpulan pasien, yang semuanya menderita penyakit yang sama. Selama pengobatan, setiap pasien menanggapi salah satu dari 5 obat, Drug A, Drug B, Drug c, Drug x and y. 
    <br>
    <br>
    Bagian dari pekerjaan Anda adalah membangun model untuk mengetahui obat mana yang mungkin cocok untuk pasien masa depan dengan penyakit yang sama. Kumpulan fitur dari kumpulan data ini adalah Age, Sex, Blood Pressure, and Cholesterol pasien, dan targetnya adalah drug yang ditanggapi oleh setiap pasien.
    <br>
    <br>
    Ini adalah contoh pengklasifikasi multikelas, dan Anda dapat menggunakan bagian pelatihan dari kumpulan data
    untuk membangun sebuah decision tree, dan kemudian menggunakannya untuk memprediksi kelas pasien yang tidak diketahui, atau meresepkannya untuk pasien baru.
</div>


baca dataset dengan pandas dataframe:

In [31]:
my_data = pd.read_csv("drug200.csv", delimiter=",")
my_data[0:5]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


<div id="practice"> 
    <h3>Practice</h3> 
   
Berapa ukuran datanya?
</div>


In [32]:
my_data.shape

(200, 6)

<div href="pre-processing">
    <h2>Pre-processing</h2>
</div>


Menggunakan <b>my_data</b> saat data Drug.csv dibaca oleh pandas, nyatakan variabel berikut: <br>

<ul>
    <li> <b> X </b> sebagai <b> Feature Matrix </b> (data of my_data) </li>
    <li> <b> y </b> sebagai <b> response vector (target) </b> </li>
</ul>



Hapus kolom yang berisi nama target karena tidak berisi nilai numerik.


In [33]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]


array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.113999999999999],
       [28, 'F', 'NORMAL', 'HIGH', 7.797999999999999],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)


Seperti yang mungkin Anda ketahui, beberapa fitur dalam kumpulan data ini bersifat categorical seperti **Sex** atau **BP**. 
Sayangnya, Sklearn Decision Trees tidak menangani variabel categorical variables. 
Tapi tetap kita bisa convert feature ini ke nilai numerik. **pandas.get_dummies()**
Convert categorical variable into dummy/indicator variables.


In [34]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]


array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.113999999999999],
       [28, 0, 2, 0, 7.797999999999999],
       [61, 0, 1, 0, 18.043]], dtype=object)


Sekarang kita bisa mengisi variabel target.


In [35]:
y = my_data["Drug"]
y[0:5]

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object

<hr>

<div id="setting_up_tree">
    <h2>Menyiapkan Decision Tree</h2>
    Kita akan menggunakan <b>train/test split</b> pada <b>decision tree</b>. mulai import <b>train_test_split</b> from <b>sklearn.cross_validation</b>.
</div>


In [36]:
from sklearn.model_selection import train_test_split

Sekarang <b> train_test_split </b> 
akan mengembalikan 4 parameter berbeda. Kami akan menamainya:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>itu <b> train_test_split </b> akan membutuhkan parameter: <br>
X, y, test_size=0.3, dan random_state=3. <br> <br>
 <b>X</b> dan <b>y</b> adalah array yang dibutuhkan sebelum split, <b>test_size</b> mewakili rasio dataset pengujian, dan <b>random_state</b> memastikan bahwa kami mendapatkan pemisahan yang sama.


In [37]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

<h3>Practice</h3>
Tampilkan shape dari X_trainset dan y_trainset. Pastikan dimensinya cocok


In [38]:
print('Shape of X training set {}'.format(X_trainset.shape),'&',' Size of Y training set {}'.format(y_trainset.shape))


Shape of X training set (140, 5) &  Size of Y training set (140,)


menampilkan shape dari X_testset dan y_testset. Pastikan dimensinya juga cocok


In [39]:
print('Shape of X training set {}'.format(X_testset.shape),'&',' Size of Y training set {}'.format(y_testset.shape))

Shape of X training set (60, 5) &  Size of Y training set (60,)


<hr>

<div id="modeling">
    <h2>Modeling</h2>
    Kami pertama-tama akan membuat instance dari<b>DecisionTreeClassifier</b> memanggil <b>drugTree</b>.<br>
    Di dalam pengklasifikasi, tentukan<i> criterion="entropy" </i> sehingga kita bisa melihat informasi gain dari setiap node.
</div>


In [40]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # it shows the default parameters

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Selanjutnya, kami akan menyesuaikan data dengan training feature matrix <b> X_trainset </b> dan training  response vector <b> y_trainset </b>


In [41]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

<hr>

<div id="prediction">
    <h2>Prediction</h2>
    mulai membuat <b>predictions</b> pada testing dataset dan simpan ke dalam variabel yang disebut <b>predTree</b>.
</div>


In [42]:
predTree = drugTree.predict(X_testset)

kita dapat menampilkan <b>predTree</b> dan <b>y_testset</b> jika ingin menampilkan bandingkan prediksi secara visual dengan nilai aktual.


In [43]:
print (predTree [0:5])
print (y_testset [0:5])


['drugY' 'drugX' 'drugX' 'drugX' 'drugX']
40     drugY
51     drugX
139    drugX
197    drugX
170    drugX
Name: Drug, dtype: object


<hr>

<div id="evaluation">
    <h2>Evaluation</h2>
    Selanjutnya import <b>metrics</b> from sklearn dan mengecek accuracy dari model.
</div>


In [44]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  0.9833333333333333


**Accuracy classification score** 
menghitung akurasi subset: kumpulan label yang diprediksi untuk sampel harus sama persis dengan kumpulan label yang sesuai di y_true.  

DI multilabel classification, fungsi mengembalikan subset accuracy. 
Jika seluruh rangkaian label yang diprediksi untuk sampel benar-benar cocok dengan true set dari label,
maka subset accuracy 1.0; selain itu 0.0.


<hr>

<div id="visualization">
    <h2>Visualization</h2>
    
    
Memulai memvisualisasi tree
</div>


In [48]:
# Notice: You might need to uncomment and install the pydotplus and graphviz libraries if you have not installed these before
#!conda install -c conda-forge pydotplus -y
#!conda install -c conda-forge python-graphviz -y

In [49]:
from  io import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [51]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = my_data.columns[0:5]
targetNames = my_data["Drug"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

InvocationException: GraphViz's executables not found

<h2>Want to learn more?</h2>

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: <a href="https://www.ibm.com/analytics/spss-statistics-software">SPSS Modeler</a>

Also, you can use Watson Studio to run these notebooks faster with bigger datasets. Watson Studio is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, Watson Studio enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of Watson Studio users today with a free account at <a href="https://www.ibm.com/cloud/watson-studio">Watson Studio</a>


### Thank you for completing this lab!

## Author

Saeed Aghabozorgi

### Other Contributors

<a href="https://www.linkedin.com/in/joseph-s-50398b136/" target="_blank">Joseph Santarcangelo</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                   |
| ----------------- | ------- | ---------- | ------------------------------------ |
| 2020-11-20        | 2.2     | Lakshmi    | Changed import statement of StringIO |
| 2020-11-03        | 2.1     | Lakshmi    | Changed URL of the csv               |
| 2020-08-27        | 2.0     | Lavanya    | Moved lab to course repo in GitLab   |
|                   |         |            |                                      |
|                   |         |            |                                      |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
